#### We will be using a college dataset (College.csv) to try to classify colleges as Private or Public based off these features:
    Private: A factor with levels No and Yes indicating private or public university
    Apps: Number of applications received
    Accept: Nunber of applications accented
    Enroll: Number of new students enrotled
    Top10perc: Pct. new students from top 10% of H.S. class
    Top25perc: Pct. new students from top 25% of HS, class
    F.Undergrad: Number of fulltine undergraduetes
    P.Undergrad: Nunber of parttime undergraduates
    Outstate: Out-of-state tuition
    Roon.Board: Room and board costs
    Books: Estimated book costs
    Personal: Estimated personal spending
    PhD: Pet. of faculty with Ph.D. 's
    Terminal: Pct. of faculty with terminal degree
    S.F.Ratio: Student/faculty ratio
    perc.alumni: Pct. alumni who donate
    Expend: Instructional expenditure per student
    Grad.Rate: Graduation rate


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [4]:
data= spark.read.csv("../../Data/College.csv", header=True, inferSchema=True)

In [5]:
data.count()

777

In [6]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [7]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

### Spark Formating of Data

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [9]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [10]:
assembler = VectorAssembler(inputCols=['Apps', 'Accept', 'Enroll', 'Top10perc', 
                             'Top25perc', 'F_Undergrad', 'P_Undergrad', 
                             'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 
                             'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'], outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
# Deal with Private column being 'yes' or 'no'
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [13]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

### The Classifiers

In [15]:
# Create all 3 models
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')
rfc = RandomForestClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')
gbt = GBTClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')

In [16]:
# Train all 3 models
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

### Model Comparison

In [17]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

##### Evaluation Metrics

In [18]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex",
                                                  predictionCol="prediction",
                                                  metricName="accuracy")

In [19]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [20]:
print("Results")
print('-'*60)
print('A single Decision Tree has an accuracy of {0:2.2f}'.format(dtc_acc*100))
print('-'*60)
print('A single Random Forest has an accuracy of {0:2.2f}'.format(rfc_acc*100))
print('-'*60)
print('An essemble using Gradient-Boosted Trees has an accuracy of {0:2.2f}'.format(gbt_acc*100))

Results
------------------------------------------------------------
A single Decision Tree has an accuracy of 92.65
------------------------------------------------------------
A single Random Forest has an accuracy of 96.33
------------------------------------------------------------
An essemble using Gradient-Boosted Trees has an accuracy of 92.65
